### GUI that uses files produced in 'create_validation_files.ipynb' for human validation of animal locations in landscape

For maximum accuracy it can be helpful to have multiple people validate the same set of images. To facilitate this, the files in the folder called 'estimated_raster_positions' generated in 'create_validation_files.ipynb' should be copied to a nwe folder called '{annoters_name}_estimated_raster_positions'. Each annotater will have their own folder where their work will be saved.

Use the notebook called 'human_groundtruth_validation.ipynb' to process the output.

### GUI Instructions 
Click in the frame called "map_image" where the highlighed animal in the frame
called "video_frame" is standing.

Press 'l' to move to the next frame
Press 'j' to go back

The position autosaves when the frame changes

In [7]:
import glob
import json
import os

import cv2

# local .json file with local path info
ungulates_root = os.path.dirname(os.path.dirname(os.getcwd()))
local_paths_file = os.path.join(ungulates_root, 'local-paths.json')
with open(local_paths_file, "r") as json_file:
    local_paths = json.load(json_file)

import sys
sys.path.append(local_paths['overhead_functions_path'])

from MapValidationGUI import MapValidationGUI

In [8]:
observation_name = "observation088"
# name of the annotater: should match the name used in the created folder for them
# (see above)
annotator_name = "koger"

# Where all the annotation folders created in 'human-groundtruth-validation.ipynb' are.
root_folder =  os.path.join(local_paths["pix4d_folder"], observation_name,
                            "validation")

In [9]:
map_images_folder = os.path.join(root_folder, 'map_images')
frame_positions_folder = os.path.join(root_folder, 'frame_positions')
video_frames_folder = os.path.join(root_folder, 'video_frames')
map_positions_folder = os.path.join(root_folder, annotator_name + '_estimated_raster_positions')
os.makedirs(map_positions_folder, exist_ok=True)

In [10]:
map_image_files = sorted(glob.glob(os.path.join(map_images_folder, '*.jpg')))
video_frames_files = sorted(glob.glob(os.path.join(video_frames_folder, '*.jpg')))
frame_positions_files = sorted(glob.glob(os.path.join(frame_positions_folder, '*.npy')))
map_positions_files = sorted(glob.glob(os.path.join(map_positions_folder, '*.npy')))

In [11]:
# How big to draw the GUI windows
max_size = 1000

gui = MapValidationGUI(map_image_files, video_frames_files, 
                       frame_positions_files, map_positions_files,
                       max_size)

In [12]:
cv2.namedWindow('map_image')
cv2.setMouseCallback('map_image', gui.clicked)

while True:
    gui.refresh_windows()
    gui.draw_frame_positions()
    gui.draw_map_positions()
    gui.show_windows()
    key = cv2.waitKey(500) & 255
    gui.react_to_keypress(key)
    if key == ord('q'):
        gui.save_map_positions()
        print('quitting')
        break
cv2.destroyAllWindows()
cv2.waitKey(1)

quitting


-1